In [12]:
import pickle

def load_model(model_name, vector_name):
    model = pickle.load(open(model_name, 'rb'))
    vectorizer = pickle.load(open(vector_name, 'rb'))
    return model, vectorizer

def predict(sentence, model, vectorizer):
    vector = vectorizer.transform([sentence])
    prediction = model.predict(vector)[0]
    return prediction

In [13]:
def predict_sentence(device, model, initial_seed, vocab_to_int, int_to_vocab, top_k=5, length=50):
    def get_word_index(w):
        if w not in vocab_to_int: 
            return 0
        return vocab_to_int[w]

    model.eval()
    # words = ['I', 'am']
    words = initial_seed.split()

    state_h, state_c = model.zero_state(1)
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for w in words:
        ix = torch.tensor([[get_word_index(w)]]).to(device)
        # print(ix)
        output, (state_h, state_c) = model(ix, (state_h, state_c))

    _, top_ix = torch.topk(output[0], k=top_k)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])

    words.append(int_to_vocab[choice])
    
#     for _ in range(length):
    while (len(words) > length and words[len(words)-1].strip().endswith('.')) == False:
        ix = torch.tensor([[choice]]).to(device)
        output, (state_h, state_c) = model(ix, (state_h, state_c))

        _, top_ix = torch.topk(output[0], k=top_k)
        choices = top_ix.tolist()
        choice = np.random.choice(choices[0])
        words.append(int_to_vocab[choice])

    return (' '.join(words))


In [14]:
import torch
import torch.nn as nn
import numpy as np

class AutoRegressive(nn.Module):
    def __init__(self, vocabulary_length, sequence_size, embedding_size, lstm_size):
        super(AutoRegressive, self).__init__()
        self.seq_size = sequence_size
        self.lstm_size = lstm_size
        self.embedding = nn.Embedding(
            vocabulary_length, 
            embedding_size
            )
        self.lstm = nn.LSTM(embedding_size,
                            lstm_size,
                            batch_first=False)
        self.dense = nn.Linear(lstm_size, vocabulary_length)
    
    def forward(self, input, previous_state ): 
        embed = self.embedding(input)
        output, state = self.lstm(embed, previous_state)
        logits = self.dense(output)

        return logits, state

    def zero_state(self, batch_size):
        return (torch.zeros(1, batch_size, self.lstm_size),
                torch.zeros(1, batch_size, self.lstm_size))


def _load_neural_network(model_name):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    checkpoint = torch.load(model_name, map_location=device)
    words = checkpoint['words']
    
    model = AutoRegressive(len(words), 32, 64, 16)
    model = model.to(device)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    int_to_vocab = checkpoint['int_to_vocab']
    vocab_to_int = checkpoint['vocab_to_int']
    
    return device, model, words, int_to_vocab, vocab_to_int

def load_story_model():
    file_name = 'story/story.pt'
    device, model, words, int_to_vocab, vocab_to_int = _load_neural_network(file_name)
    
    def run_prediction(initial_words, min_length=50):
        return predict_sentence(device, model, initial_words, vocab_to_int, int_to_vocab, length=min_length)
    
    return run_prediction



In [15]:
# predict = load_story_model()
# predict('queen grabbed')

In [16]:
def _load_model_predict(model, vector):
    m, v = load_model(model, vector)
    
    return lambda sentence: predict(sentence, m, v)
    
def load_model1():
    model = 'model1/model.pickle'
    vector = 'model1/vector.pickle'
    return _load_model_predict(model, vector)

def load_model2():
    model = 'model2/model.pickle'
    vector = 'model2/vector.pickle'
    return _load_model_predict(model, vector)

In [17]:
# https://stanfordnlp.github.io/stanfordnlp/
# !pip install stanfordnlp

import stanfordnlp
# stanfordnlp.download('en')   # This downloads the English models for the neural pipeline
nlp = stanfordnlp.Pipeline() # This sets up a default neural pipeline in English


Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/Users/msarica/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/Users/msarica/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/Users/msarica/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/Users/msarica/stanfordnlp_resources/en_ewt_models/en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/Users/msarica/stanfordnlp_resources/en_ewt_models/en_ewt_parser.pt', 'pretrain_path': '/Users/msarica/stanfordnlp_resources/en_ew

In [18]:
import re

def generate_time_units():
    time_units = [
        ('second', 1),
        ('minute', 60),
        ('hour', 3600),
        ('day', 3600 * 24)
    ];

    times = []
    for word, unit in time_units:
        s = '\w+(?=\s'+ word +')'
        r = re.compile(s, re.IGNORECASE)
        times.append((r, unit))
    return times

times = generate_time_units()

def timer(sentence):
    def parse_int(value):
        try:
            return int(value.strip())
        except: 
            return 0
        
    def convert_to_sec(regex_result, value_to_sec):
        if regex_result == None:
            return 0
        
        value = parse_int(regex_result[0])
        return value * value_to_sec
    
    total = 0
    for r, unit in times:
        m = r.search(sentence)
        total += convert_to_sec(m, unit)
        
    if total == 0:
        m = re.search(r'\d+', sentence)
        total += convert_to_sec(m, 1)
        
    return total

# timer('set a timer for 3 minutes 30 seconds')
# timer('sadfa 5')

In [19]:
def get_object(sentence, nlp=nlp):
    doc = nlp(sentence)
    # doc.sentences[0].print_dependencies()

    objs = []
    for i in doc.sentences[0].words:
        # print(i.dependency_relation)
        if(i.dependency_relation == 'obj'):
            objs.append(i.text)
    
    return objs
    

# get_object('sophia set a timer for 5 minutes')

In [20]:
def tell_story(sentence):
    m = re.search(r'(?<=about\s)[\w\s]*', sentence)
    
    initial = m[0]
    return story_model(initial)
    
# tell_story('tell me a story about snow white')

In [21]:
from datetime import datetime

def time_now():
    now = datetime.now()
    current_time = now.strftime("%I:%M %p")
    return f"it is {current_time}"

# time_now()

In [22]:
pred1 = load_model1()
predict_category = load_model2()

story_model = load_story_model()

In [23]:
def process(sentence):
    # predict category
    category = predict_category(sentence)
    print(category)
    
    actions = {
        'timer': lambda sent: timer(sent),
        'shopping': lambda sent: get_object(sent),
        'time_now': lambda sent: time_now(),
        'story': lambda sent: tell_story(sent)
    }
    
    if category in actions:
        text = actions[category](sentence)
    else:
        text = None
    
    return category, text
    
# process('sophia can you add milk to my list')
# process('set a timer for 5 ')
# process('what time is it')
# process('sophia please tell me a story about a witch')

In [ ]:
from flask import Flask
from flask import jsonify, request
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.route('/')
def start():
    sentence = request.args.get('sentence')
    print(sentence)

    category, text = process(sentence)
    print(category, text)
    return jsonify({
        'category': category,
        'text': text
    })
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Aug/2020 22:37:23] "OPTIONS /?sentence=Sophia%20tell%20me%20a%20story%20about%20my%20son HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2020 22:37:23] "GET /?sentence=Sophia%20tell%20me%20a%20story%20about%20my%20son HTTP/1.1" 200 -


Sophia tell me a story about my son
story
story my son like no peace day heart in her the wild forest, the poor proof that hour, queen as white looked more at all the lung in front a token. i a beautiful woman, soon took them lady thou window at all or night. the looking-glass snow, answered, thou, thou, spoke looking-glass, and pride beautiful snow, as red snow, snow, she began that as proof had passed she pricked he no that it seemed and said, ah dear run away, to kill more come looking-glass, snow, she called a child the wild never home queen.


127.0.0.1 - - [03/Aug/2020 22:38:30] "OPTIONS /?sentence=Sophia HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2020 22:38:30] "GET /?sentence=Sophia HTTP/1.1" 200 -


Sophia
timer
timer 0


...

Sofia set a timer for 5 minutes
timer
timer 300


127.0.0.1 - - [03/Aug/2020 22:44:46] "OPTIONS /?sentence=Sofia%20add%20bread%20to%20my%20shopping%20list HTTP/1.1" 200 -
/Users/distiller/project/conda/conda-bld/pytorch_1591914925853/work/aten/src/ATen/native/TensorAdvancedIndexing.cpp:573: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
127.0.0.1 - - [03/Aug/2020 22:44:46] "GET /?sentence=Sofia%20add%20bread%20to%20my%20shopping%20list HTTP/1.1" 200 -


Sofia add bread to my shopping list
shopping
shopping ['bread']


127.0.0.1 - - [03/Aug/2020 22:45:03] "OPTIONS /?sentence=Sophia%20tell%20a%20story%20about%20my%20beautiful%20wife HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2020 22:45:03] "GET /?sentence=Sophia%20tell%20a%20story%20about%20my%20beautiful%20wife HTTP/1.1" 200 -


Sophia tell a story about my beautiful wife
story
story my beautiful wife back my sight. kill her, envy. in her finger day have run away needful queen. front thou, pretty him looking-glass, to salt them, a token. and said, ah soon into if eaten the lung in her in my life. and three its did not and liver of it and looked at so looking-glass, on the queen looked pretty now thought to salt in in her away forest, no beautiful if a stone had soon after himself fell eaten terrified had looking-glass, looking-glass, looking-glass, of snow-white.
